In [1]:
import os
!pip install tqdm
!pip install sentence-transformers
!pip install -U torch
!pip install PyMuPDF
!pip install accelerate
!pip install bitsandbytes
!pip install flash-attn --no-build-isolation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 42.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187309225 sha256=237ef9c6157db394e1ddde4ba609a21ebb98382377a27041edc09318801a6f24
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash-attn


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os
import shutil

# drive.mount('/content/drive')
# pdf_drive_path = '/content/drive/MyDrive/RAG_model/RAG_model.pdf'
# pdf_path ='./RAG_model.pdf'

# if not os.path.exists(pdf_drive_path):
#     print("File doesn't exist in Google Drive.")
# else:
#     shutil.copy(pdf_drive_path, pdf_path)
#     print(f"File copied to Colab environment as {pdf_path}")

# if os.path.exists(pdf_path):
#     print(f"File {pdf_path} exists in Colab.")
# else:
#     print("File copy failed.")

In [ ]:
import fitz
from tqdm.auto import tqdm

def text_formatter(text: str) -> str:
    cleaned_text = text.replace("\n", " ").strip()
    return cleaned_text

def open_and_read_pdf(pdf_path: str) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):
        text = page.get_text()
        text = text_formatter(text)
        pages_and_texts.append({"page_number": page_number+1,
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4,
                                "text": text})
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': 1,
  'page_char_count': 2834,
  'page_word_count': 485,
  'page_sentence_count_raw': 17,
  'page_token_count': 708.5,
  'text': 'The Rise of Alessandro Vargas – Manchester United’s New Star  In the summer of 2027, Manchester United shocked the football world by signing a relatively  unknown but immensely talented young midfielder, Alessandro Vargas, from Sporting  Lisbon for a fee of £55 million. Born in Porto but of Chilean descent, the 21-year-old had  only two seasons of professional football under his belt before the Red Devils swooped in to  secure his signature. While his name wasn’t widely known at the time, insiders at Old  Trafford were convinced that Vargas was destined to become the future heartbeat of the  Manchester United midfield.  The Announcement  The club’s official website announced the signing in early August, with a glowing  endorsement from Erik ten Hag: “Alessandro Vargas is one of the most exciting young  midfielders in Europe. His intelligence 

In [ ]:
import pandas as pd
df = pd.DataFrame(pages_and_texts)
df.head()
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,13.00,13.00,13.00,13.00,13.00
mean,7.00,2760.31,484.46,21.46,690.08
std,3.89,166.43,31.81,2.76,41.61
min,1.00,2471.00,435.00,17.00,617.75
25%,4.00,2627.00,457.00,19.00,656.75
50%,7.00,2793.00,485.00,23.00,698.25
75%,10.00,2898.00,509.00,23.00,724.50
max,13.00,2964.00,536.00,26.00,741.00


In [ ]:
from spacy.lang.en import English
nlp = English()
nlp.add_pipe("sentencizer")
for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy
count,13.00,13.00,13.00,13.00,13.00,13.00
mean,7.00,2760.31,484.46,21.46,690.08,23.15
std,3.89,166.43,31.81,2.76,41.61,3.44
min,1.00,2471.00,435.00,17.00,617.75,17.00
25%,4.00,2627.00,457.00,19.00,656.75,20.00
50%,7.00,2793.00,485.00,23.00,698.25,24.00
75%,10.00,2898.00,509.00,23.00,724.50,26.00
max,13.00,2964.00,536.00,26.00,741.00,27.00


In [ ]:
num_sentence_chunk_size = 10

def split_list(input_list: list,
               slice_size: int) -> list[list[str]]:
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy,num_chunks
count,13.00,13.00,13.00,13.00,13.00,13.00,13.00
mean,7.00,2760.31,484.46,21.46,690.08,23.15,2.69
std,3.89,166.43,31.81,2.76,41.61,3.44,0.48
min,1.00,2471.00,435.00,17.00,617.75,17.00,2.00
25%,4.00,2627.00,457.00,19.00,656.75,20.00,2.00
50%,7.00,2793.00,485.00,23.00,698.25,24.00,3.00
75%,10.00,2898.00,509.00,23.00,724.50,26.00,3.00
max,13.00,2964.00,536.00,26.00,741.00,27.00,3.00


In [ ]:
import re
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)
        chunk_dict["sentence_chunk"] = joined_sentence_chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4
        pages_and_chunks.append(chunk_dict)
len(pages_and_chunks)

  0%|          | 0/13 [00:00<?, ?it/s]

35

In [ ]:
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,35.00,35.00,35.00,35.00
mean,7.26,1011.26,166.57,252.81
std,3.56,374.98,60.39,93.74
min,1.00,131.00,20.00,32.75
25%,4.50,894.50,154.00,223.62
50%,7.00,1109.00,183.00,277.25
75%,10.00,1216.50,195.00,304.12
max,13.00,1621.00,262.00,405.25


In [ ]:
min_token_length = 30
filtered_df = df[df["chunk_token_count"] <= min_token_length]
if not filtered_df.empty:
    sample_size = min(5, len(filtered_df))
    for row in filtered_df.sample(sample_size).iterrows():
        print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')
else:
    print("No rows found with token count less than or equal to 30.")


No rows found with token count less than or equal to 30.


In [ ]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'page_number': 1,
  'sentence_chunk': 'The Rise of Alessandro Vargas – Manchester United’s New Star In the summer of 2027, Manchester United shocked the football world by signing a relatively unknown but immensely talented young midfielder, Alessandro Vargas, from Sporting Lisbon for a fee of £55 million. Born in Porto but of Chilean descent, the 21-year-old had only two seasons of professional football under his belt before the Red Devils swooped in to secure his signature. While his name wasn’t widely known at the time, insiders at Old Trafford were convinced that Vargas was destined to become the future heartbeat of the Manchester United midfield. The Announcement The club’s official website announced the signing in early August, with a glowing endorsement from Erik ten Hag: “Alessandro Vargas is one of the most exciting young midfielders in Europe. His intelligence on the ball, work ethic, and ability to control the tempo of the game make him an ideal fit for our long-term projec

In [ ]:
# Import required libraries
import time
from sentence_transformers import SentenceTransformer

# Load the model and move it to CPU
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", device="cpu")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Time the execution of the embedding creation
%%time

# Move the model to CPU
embedding_model.to("cpu")

# Loop through the items and compute embeddings
for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

  0%|          | 0/35 [00:00<?, ?it/s]

CPU times: user 17.8 s, sys: 57.8 ms, total: 17.9 s
Wall time: 18.4 s


In [11]:
# from git_setup import configure_git, change_directory, create_gitignore, initialize_repo, add_files, commit_files, push_to_github
# from my_key import my_key

# configure_git("Irfan-Hamid", "irfanhamid19@gmail.com")
# change_directory()
# create_gitignore()
# initialize_repo()
# add_files()
# commit_files("Initial commit: Added notebook and PDF files")
# push_to_github(my_key, 'github.com/your_github_username/your_repo_name.git')


In [7]:
%cd /content/drive/MyDrive/RAG_model
from git_setup import configure_git, change_directory, create_gitignore, initialize_repo, add_files, commit_files, push_to_github
from my_key import my_key

/content/drive/MyDrive/RAG_model


In [ ]:
configure_git("Irfan-Hamid", "irfanhamid19@gmail.com")
add_files()
commit_files("fourth commit: code updation")
push_to_github(my_key, 'github.com/Irfan-Hamid/LLM_RAG_IMPLEMENTATION.git')

In [5]:
! git status

fatal: not a git repository (or any of the parent directories): .git


In [4]:
print("check2")

check2
